In [11]:
import os  
from langchain_google_genai import ChatGoogleGenerativeAI  
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate 
from langgraph.graph import StateGraph, START, END 
from typing import Optional, List, Dict, Any, TypedDict, Literal  
from dotenv import load_dotenv   
import pandas as pd  
import numpy as np 
import nest_asyncio 
import asyncio 
import logging 


load_dotenv() 


nest_asyncio.apply() 




In [ ]:
from langchain_google_vertexai import ChatVertexAI  
from langchain_google_vertexai.model_garden import ChatAnthropicVertex  






anthropic_vertex = ChatAnthropicVertex(model="claude-3-5-sonnet-20240620")  


print(anthropic_vertex)











client=<anthropic.lib.vertex._client.AnthropicVertex object at 0x00000222FF72C9E0> async_client=<anthropic.lib.vertex._client.AsyncAnthropicVertex object at 0x000002229A41ACF0> project='animated-graph-463101-t5' model_name='claude-3-5-sonnet-20240620' client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None} default_metadata=() model_kwargs={}


In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=os.getenv("GOOGLE_FREE_API_KEY"))  













In [ ]:
from __future__ import annotations

from typing import Any, Dict, List, Optional, Literal

from pydantic import BaseModel, Field
from langchain_core.tools import tool


# ---------------------------
# Pydantic schemas
# ---------------------------
class AttributionQuote(BaseModel):
    text: str = Field(..., description="Short quote from summary/transcript.")
    timestamp: Optional[str] = None
    speaker: Optional[str] = None
    chunk_id: Optional[str] = None


class ProductItem(BaseModel):
    name: Optional[str] = None
    description: Optional[str] = None
    features: List[str] = Field(default_factory=list)
    protocols: List[str] = Field(default_factory=list)
    benefits_as_stated: List[str] = Field(default_factory=list)
    attribution: List[AttributionQuote] = Field(default_factory=list)


class ProductSummary(BaseModel):
    products: List[ProductItem] = Field(default_factory=list)


class TreatmentItem(BaseModel):
    name: Optional[str] = None
    procedure_or_protocol: List[str] = Field(default_factory=list)
    outcomes_as_reported: List[str] = Field(default_factory=list)
    risks_or_contraindications: List[str] = Field(default_factory=list)
    confidence: Literal["high", "medium", "low"] = "medium"
    attribution: List[AttributionQuote] = Field(default_factory=list)


class MedicalSummary(BaseModel):
    treatments: List[TreatmentItem] = Field(default_factory=list)


class SectionItem(BaseModel):
    title_or_topic: Optional[str] = None
    approx_time_range: Optional[str] = Field(
        None, description="e.g., 00:00–05:30"
    )


class HighLevelOverview(BaseModel):
    purpose: Optional[str] = None
    participants: List[str] = Field(default_factory=list)
    main_sections: List[SectionItem] = Field(default_factory=list)
    key_takeaways: List[str] = Field(default_factory=list)
    attribution: List[AttributionQuote] = Field(default_factory=list)


class ClaimItem(BaseModel):
    speaker: Optional[str] = None
    claim_text: str
    claim_type: Literal["causal", "quantitative", "experiential", "other"] = "other"
    evidence_present_in_transcript: Literal["yes", "no"] = "no"
    attribution: List[AttributionQuote] = Field(default_factory=list)


class ClaimsSummary(BaseModel):
    claims: List[ClaimItem] = Field(default_factory=list)


class CompanyItem(BaseModel):
    canonical_name: str
    aliases: List[str] = Field(default_factory=list)
    description: Optional[str] = None


class CompaniesSummary(BaseModel):
    entities: List[CompanyItem] = Field(default_factory=list)





def _to_dict(model: BaseModel) -> Dict[str, Any]:
    # pydantic v2 / v1 compatibility
    return model.model_dump() if hasattr(model, "model_dump") else model.dict()




@tool(
    "submit_product_information",
    args_schema=ProductSummary,
    return_direct=True,
    parse_docstring=True,
    description="Submit structured product information"
)
def submit_product_information(products: List[ProductItem]) -> Dict[str, Any]:
    """
    Provide structured product information. This tool echoes the provided arguments unchanged.

    Guidance:
    - Use ONLY context content; no external facts.
    - Include product name, description, features, protocols, benefits.
    - Add AttributionQuote items for quotes/timestamps when present.

    Args:
        products: List of product items.

    Returns:
        Dict matching ProductSummary.
    """
    return _to_dict(ProductSummary(products=products))


@tool(
    "submit_medical_treatment",
    args_schema=MedicalSummary,
    return_direct=True,
    parse_docstring=True,
    description="Submit structured medical treatment information"
)
def submit_medical_treatment(treatments: List[TreatmentItem]) -> Dict[str, Any]:
    """
    Provide structured medical treatment information. This tool echoes the provided arguments unchanged.

    Guidance:
    - Use ONLY context content; no external facts.
    - Include name, procedure_or_protocol, outcomes_as_reported, risks_or_contraindications.
    - Set confidence to high/medium/low as appropriate.
    - Add AttributionQuote items when available.

    Args:
        treatments: List of treatments.

    Returns:
        Dict matching MedicalSummary.
    """
    return _to_dict(MedicalSummary(treatments=treatments))


@tool(
    "submit_high_level_overview",
    args_schema=HighLevelOverview,
    return_direct=True,
    parse_docstring=True,
    description="Submit a high-level overview"
)
def submit_high_level_overview(
    purpose: Optional[str] = None,
    participants: List[str] = [],
    main_sections: List[SectionItem] = [],
    key_takeaways: List[str] = [],
    attribution: List[AttributionQuote] = [],
) -> Dict[str, Any]:
    """
    Provide a high-level overview. This tool echoes the provided arguments unchanged.

    Guidance:
    - Use ONLY context content; no external facts.
    - Populate purpose, participants, main_sections (with approx_time_range if present), key_takeaways.
    - Include AttributionQuote items when appropriate.

    Args:
        purpose: Overall purpose.
        participants: Key participants.
        main_sections: Main sections/topics (with approx_time_range if available).
        key_takeaways: 5–7 key takeaways when available.
        attribution: Supporting quotes and timestamps.

    Returns:
        Dict matching HighLevelOverview.
    """
    return _to_dict(
        HighLevelOverview(
            purpose=purpose,
            participants=participants,
            main_sections=main_sections,
            key_takeaways=key_takeaways,
            attribution=attribution,
        )
    )


@tool(
    "submit_claims_made",
    args_schema=ClaimsSummary,
    return_direct=True,
    parse_docstring=True,
    description="Submit explicit claims"
)
def submit_claims_made(claims: List[ClaimItem]) -> Dict[str, Any]:
    """
    Provide explicit claims. This tool echoes the provided arguments unchanged.

    Guidance:
    - Use ONLY context content; no external facts.
    - For each claim, include speaker (if known), claim_text (≤20 words if possible),
      claim_type (causal/quantitative/experiential/other), evidence_present_in_transcript (yes/no),
      and any AttributionQuote items.

   Args:
        claims: List of claim items.

    Returns:
        Dict matching ClaimsSummary.
    """
    return _to_dict(ClaimsSummary(claims=claims))


@tool(
    "submit_businesses_entities",
    args_schema=CompaniesSummary,
    return_direct=True,
    parse_docstring=True, 
    description="Submit businesses/organizations/entities"
)
def submit_businesses_entities(entities: List[CompanyItem]) -> Dict[str, Any]:
    """
    Provide businesses/organizations/entities. This tool echoes the provided arguments unchanged.

    Guidance:
    - Use ONLY context content; no external facts.
    - For each entity, include canonical_name and any aliases; add a concise description of the role/relevance if present.

    Args:
        entities: List of entities.

    Returns:
        Dict matching CompaniesSummary.
    """
    return _to_dict(CompaniesSummary(entities=entities))


TOOLS = [

    submit_product_information,
    submit_medical_treatment,
    submit_high_level_overview,
    submit_claims_made,
    submit_businesses_entities,
] 

MODEL_MAP = {
    "product_information": ProductSummary,
    "medical_treatment": MedicalSummary,
    "high_level_overview": HighLevelOverview,
    "claims_made": ClaimsSummary,
    "businesses_entities": CompaniesSummary,
}


TOOL_MAP = { 
    "product_information": submit_product_information,
    "medical_treatment": submit_medical_treatment,
    "high_level_overview": submit_high_level_overview,
    "claims_made": submit_claims_made,
    "businesses_entities": submit_businesses_entities,
}





In [14]:
test_tool = """
The RevitaGlow Pro 3000 is our latest red light therapy device that uses patented PhotoMax™ technology. Operating at 660nm and 850nm wavelengths, it delivers targeted photobiomodulation through 200 medical-grade LEDs. The sleek panel design provides coverage for face and body treatments in just 10-20 minute sessions.

The OxyBoost Elite System combines exercise with oxygen therapy (EWOT) through our proprietary OxyFlow™ delivery method. The system includes a high-flow oxygen concentrator producing 95% pure O2, paired with our smart mask technology that automatically adjusts oxygen levels based on exercise intensity. The wheeled stand and ergonomic mask design allow for comfortable use with any cardio equipment."""






In [3]:
transcript_text = """ 

EP_1301_BRAD_PITZELE_AUDIO

Dave: [00:00:00] Increasing metabolic demand in a very high oxygen environment has unique [00:00:05] metabolic benefits that turn off, in some cases, tons of autoimmune inflammation. Every [00:00:10] year

Brad: past the age of 25, you lose 1% of your ability [00:00:15] to utilize oxygen. One of the major reasons for this is inflammation with. In the [00:00:20] capillaries, inflammation is at the root of basically every chronic health condition we have out there.

Host: [00:00:25] Brad Ley, founder of 1000 Roads. After being diagnosed with metastatic [00:00:30] melanoma and multiple autoimmune conditions, Brad discovered exercise with [00:00:35] oxygen therapy and restored his health. Now he's made this powerful healing tool accessible [00:00:40] for home use, helping thousands of others on their healing journeys.

Brad: There's a lot of [00:00:45] research out. There that shows anywhere in your body you have low [00:00:50] oxygen, you have inflammation, and anywhere you have inflammation, you have low oxygen. How

Dave: did you [00:00:55] get into this strange idea that you should breathe a lot of oxygen while you're exercising? [00:01:00] You are listening to the human upgrade with Davey.[00:01:05]

You might have heard me talk about being 300 pounds in my [00:01:10] early twenties. And I did everything you could possibly do. And [00:01:15] I went really deep on all the bizarre stuff. And I was living in a small farming town in [00:01:20] Northern California and I went to the 24 Hour Fitness, and it was called Nautilus back [00:01:25] then.

And I said, I'm gonna work out six days a week, 90 minutes a day, no matter [00:01:30] what. It doesn't matter if I'm tired, I'm sick, I'm just gonna go. And it's gonna be non-negotiable 'cause I have to lose this a [00:01:35] hundred pounds. I do not want another knee surgery. So one of the things I learned [00:01:40] about was something called e watt.

It was very unknown. [00:01:45] This would've been 1995, maybe. Uh, something like that, maybe even [00:01:50] 1994. And. I was like, how do I do this? I went to Craigslist and I bought an [00:01:55] oxygen cylinder that was full of oxygen, 'cause medical oxygen is very well controlled. And I [00:02:00] went to the gym and I brought in this big oxygen cylinder and I got on the treadmill [00:02:05] and I put the oxygen mask on, like I had emphysema and I'm this, you know, fat [00:02:10] early twenties kid on the treadmill, sweating my butt off, breathing through an oxygen mask and thinking [00:02:15] I must be the stupidest looking person in the gym.

A little did I know that I was just the coolest, biohacker, [00:02:20] just, uh, I was ahead of my time maybe, but that was my first, uh, learning [00:02:25] about e watt or exercise with oxygen, oxygen therapy, and it's something I've experimented [00:02:30] with ever since. We have a similar technology at Upgrade labs today, so I wanted to go deep [00:02:35] with you on this episode and bringing an expert who's really, really learned what [00:02:40] happens when we.

Change the intensity of what goes into our bodies and change the type of [00:02:45] oxygen. This is not hyperbaric. It's something different, and I think it's gonna be a really [00:02:50] fascinating interview about mitochondria, metabolism, respiration, and all the other [00:02:55] cool stuff that underlies everything we do as biohackers and as human beings.

Our [00:03:00] guest today is Brad Ley, and he is. By, [00:03:05] we'll say by necessity, decided to learn about ott. Brad, [00:03:10] welcome to the show. Hey, thanks so much Dave. I appreciate it. You've been talking about this relatively [00:03:15] unknown mitochondrial hack. Uh, you've been on Born to Heal and Tick Bootcamp, [00:03:20] and you started building OTT systems and started your company [00:03:25] 1000 Roads back in 2018.

How did you get [00:03:30] into this strange idea that you should breathe a lot of oxygen while you're exercising?

Brad: You're right. Uh, [00:03:35] necessity is the mother of all invention, and mine was nonetheless. Um, [00:03:40] I had weird autoimmune conditions my whole life. And then [00:03:45] in 2010 timeframe I was diagnosed with autoimmune [00:03:50] arthritis, and it started to get progressively worse.

They put me on all the [00:03:55] drugs and they would help for a while, and then when they stopped working six months later, I'd be in a worse [00:04:00] condition. A lot of those drugs, they they suppress your immune system. 'cause your immune system causes [00:04:05] all the symptoms of the autoimmunity. So the theory is suppress the immune system and then the [00:04:10] symptoms go away.

And I was doing that and like I said, I'd be better and then I'd [00:04:15] step two more steps down after they'd stop working. And then all of a [00:04:20] sudden I, I had this weird little patch on my shoulder and I went and saw the dermatologist and it ended up [00:04:25] being melanoma. And this a side effect of, of. [00:04:30] The medications is they can cause a high risk of cancer and specifically [00:04:35] melanoma.

So I went back to my rheumatologist and I said, so what do I do here, doc, if I take, [00:04:40] keep taking these meds, I'm gonna get cancer and die of it. But if I don't take him, I'm [00:04:45] gonna die of the autoimmunity. I'm like, you know, I'm a I at this time I was a father of three children and I was like, [00:04:50] I, I, you know, I can't die.

That's not an option. So [00:04:55] he didn't have much of a really good answer for me. It was like, Hey, we're just gonna give you more of the [00:05:00] same. And may, you know, kind of hope for the best. And so that started me down the rabbit hole of [00:05:05] alternate wellness practices and trying to find another way out to your point, like there was, [00:05:10] there was no traditional medical way out.

And so finally I [00:05:15] got sicker and sicker, and it was probably in about 2016, I went and saw this, [00:05:20] um, this. This doctor who practiced alternate health practice and he said, Brad, you're [00:05:25] in a really bad way and you need to try something. You either need to try hyperbaric [00:05:30] oxygen. Or you need to try exercise with oxygen therapy.

And I had never heard of [00:05:35] it. Kind of like you, like it was, it's no one had heard of it. And so I [00:05:40] immediately looked into hyperbaric oxygen, but it was very expensive and very time [00:05:45] consuming, and I was already kind of like fighting for my job. I didn't have any energy. I had brain fog. I [00:05:50] had pain all over.

Really bad pain and I just knew I didn't have an [00:05:55] hour and a half to spend inside of a tube and I didn't have the 75 grand. We were talking the [00:06:00] hard shell hyperbaric, like the intense stuff. And I didn't have that kind of money. [00:06:05] So I was like, well, let me look at this EWT thing. And, um, [00:06:10] started to research it.

Sounded promising, but there wasn't much on the internet. There weren't people [00:06:15] talking about how they were using it and getting better. Um, there was only the company selling it, and a lot [00:06:20] of the, a lot of it was marketing material that quite frankly had a lot of hyperbole in [00:06:25] it. And I just, I didn't have a lot of confidence and they were pretty high priced and I just [00:06:30] wasn't willing to spend $10,000 on an experiment when every experiment I had tried had [00:06:35] failed.

So. I always joke, I'm a recovering engineer. I started my career in [00:06:40] engineering and so I, um, at this point I was no longer in [00:06:45] engineering, but I said to put that degree back to work and I kind of just designed my own system [00:06:50] and lo and behold, and I, a year later I saw that same doctor and I was getting better and he said, [00:06:55] oh, are you doing the oxygen?

And I'm like, I am. And he said, well, you know. [00:07:00] Who'd you buy it from? So I can tell my other patients about it. And I said, I actually just made my own [00:07:05] doc. And he looked at me like I was crazy. And then he said, after apologies, he's like, [00:07:10] would you consider making 'em for my patients? And that was kind of the birth of the idea.

It was like, well, I'd [00:07:15] never thought about it, but I had, I came in at, you know, it was kind of [00:07:20] like this. Wanting to pay back because I had felt taken advantage of by so many [00:07:25] wellness practices where they wanted me to pay 25 grand or more to [00:07:30] experiment with this modality or that or, and it just felt like it's very difficult [00:07:35] when you're in a chronic health condition.

And you're outside the medical standard. You, you yeah. [00:07:40] Mold and

Dave: Lyme and all, all that stuff. Same. Same with me. There's a reason I say I spent two and a half million [00:07:45] dollars reversing my age. The first half a million dollars was just getting well from chronic autoimmunity and [00:07:50] toxic mold and all this stuff, right?

And, and so I'm lucky [00:07:55] I made enough money in my twenties despite my brain being cooked, um, that I could do that. And [00:08:00] without that, I think I would have. Had lupus or ms, and certainly I'd be disabled. [00:08:05] So that's one of the things that motivates me to have you on the show is like, let's tell everybody about [00:08:10] this.

And like, you, like I, I think it's kind of ridiculous. There's one [00:08:15] vendor of OT. Charge a huge amount of money for basically a big trash bag. [00:08:20] And it's, it's just like, not, not the way I would go about doing it. What I wanna know [00:08:25] is, uh, and by the way, this is full disclosure. I've had a soft sided chamber for 18 years [00:08:30] hyperbaric, and I have a hard sided chamber downstairs.

I'm a fan of hyperbaric and I'm a fan of EW Watt. [00:08:35] Um, and in fact, I'm a ch, I'm a fan of even, um, doing the intermittent versions of it. [00:08:40] So. I mean, you could have gone out there and spent 10 grand on a big trash bag, but why didn't you, you [00:08:45] know, what, what, what made you decide to build it yourself?

Brad: Well, the first thing was I just didn't [00:08:50] believe in it.

Be honest, I didn't believe in it. 'cause I have tried everything at this point. I mean, [00:08:55] diets, supplements, all sorts of modalities, you know, and, and at, at one [00:09:00] point the doctor said, oh, I'm gonna give you the big guns. We're gonna do ozone treatment. And by the way, all these things are great [00:09:05] things and I'm not putting 'em down.

They just didn't work for me. So when I was looking at, you know, [00:09:10] 10,000 or plus dollars, it, it just wasn't palpable for me. To [00:09:15] spend that kind of money on an experiment that like at the end of it, I was gonna make no better [00:09:20] now. Now what do I do? So I started researching it and kind of to your point, [00:09:25] like I, I knew I could, I could do this better.

And I think there's a couple things [00:09:30] I saw wrong with the industry. One was, it was a lot of hyperbole and I just wish someone [00:09:35] would bring me some science. I wish someone would bring me. Real people [00:09:40] talking about it so that I could hear from other people using it and have some confidence in it. You know, the market [00:09:45] back then, like when you started the market, was primarily these companies selling to [00:09:50] chiropractor's, offices and wellness clinics.

That's a small universe. And [00:09:55] so it makes sense that they had to charge high prices in order to make it a viable [00:10:00] business. And my thought was, well, what if we just, we just trashed that price and brought it way [00:10:05] down? Then we could extend this out to people. And the other thing [00:10:10] about it is the most effective oxygen therapy, the most effective [00:10:15] wellness modality, regardless, is the one you can do consistently over time.[00:10:20]

And so. If you can, you know, and that's what I loved about this. It was [00:10:25] 15 minutes a day is all it takes to do ot. And if you could bring the price down so people could do it in their [00:10:30] home, they didn't need to go to a center. That's something most of us can do. Um, [00:10:35] and you know this Dave, like when you're chronically ill and you're taking the supplements and doing all the [00:10:40] modalities, every one of 'em only takes you 25 minutes counting pills and this until you're [00:10:45] like, holy cow.

My entire calendar is all this stuff all day and I feel great for that last [00:10:50] 10 minutes and then back to bed and back to square one. So it's like, I was called the [00:10:55] ceremonies and so like, there's too many ceremonies when you're, when you're sick, trying to get well. And [00:11:00] so anything that was, you could do in a very quick.

Period of time and it would had multiple [00:11:05] positive impacts. That was amazing for me.

Dave: Now you understand why Upgrade labs [00:11:10] exists for if you're a new listener. This is my franchise company where we have about a dozen [00:11:15] biohacking modalities in the same place for a single membership fee because I had to build a million [00:11:20] dollar biohacking lab in my barn because I don't have time to drive everywhere, or you just couldn't get it where I lived.[00:11:25]

And so like, no one should have to do that. And the idea that the thing you need to do every day, [00:11:30] just do it at home. And, and I also wanna point out, Brad, a lot of the biohacks that [00:11:35] we're using, they go back at least to Tesla. Like if we're looking at electrical things or some of the [00:11:40] PEMF and some of it much longer, like Pliny, the elder [00:11:45] from way back in the day was using electric eels to cure migraines, right?

So [00:11:50] we, we have this long history of biohacking, but Iwat only was [00:11:55] invented in East Germany in the 1960s. Right. So this is a very [00:12:00] new thing that just didn't exist. In fact, hyperbaric has existed longer that, that's why it's so unknown. It takes a long time for [00:12:05] things that work to get out there. And so it was invented in the sixties, but the [00:12:10] book on it only came out in 1990, right?

I was doing it in 1994 because I know I got a [00:12:15] futurist or something. I, I, I found out about it from the longevity nonprofit stuff I was doing. Said I [00:12:20] gotta try this, and I ran out of oxygen and found I could not refill it. [00:12:25] Oxygen is incredibly restricted here in the system that you make. Where'd you get the

Brad: [00:12:30] oxygen?

That's a great question. So we use a device called an oxygen concentrator, and [00:12:35] effectively what it does is it takes the air, your breathing, that's at 21% oxygen and it [00:12:40] purifies it to 93% oxygen. 93 is the medical spec they [00:12:45] use. So these are the same sorts of machines that people have, like COPD, [00:12:50] uh, or emphysema would use in their home when they need supplemental oxygen.

Dave: Uh, it's exactly. [00:12:55] How to do it. By the way, anyone who had one of your systems during. [00:13:00] The pandemic had oxygen. Right? So when, when it looked like there was a lung issue, I'm like, well, I have two oxygen [00:13:05] concentrators at home. And I just bought all the welding oxygen that was around me. So I'm like, I have enough oxygen to treat my [00:13:10] whole neighborhood if, if they're, if, if they need it.

Because gonna, the hospital probably is a good way to die at the [00:13:15] time.

Brad: Yeah. And these machines, they just last, I mean, if you get an oxygen concentrator [00:13:20] for, for the purpose of wot, because we're only using it a small number of hours a day, it, for it to [00:13:25] last a decade is not, is nothing the bad an eyelash yet?[00:13:30]

Dave: The idea here is understand changing [00:13:35] the amount of oxygen that you breathe during intense exercise. It's a big deal. [00:13:40] Right.

Brad: Why does it work? That's an awesome question. So as you mentioned, there was a [00:13:45] gentleman named Van Arden in the 1960s and seventies who did some research, and [00:13:50] he's the same guy who invented the scanning electron microscope.

He's about 600 patents to [00:13:55] his name, just prolific inventor, super smart. He was in part of the commercialization of the [00:14:00] first CRT TV sets in the 1930s, so really smart guy. He got [00:14:05] very interested in oxygen and particularly oxygen for anti-cancer purposes, and [00:14:10] he started doing this research. And so here's the thing that happens every [00:14:15] year past the age of 25, you lose 1% of your ability [00:14:20] to utilize oxygen, and most of us don't like notice this.

[00:14:25] Right off the bat. So 26 and 30, you don't necessarily notice it, but that's the reason why [00:14:30] you're at a lower energy state when you're in your sixties than you were in your twenties. [00:14:35] And so what he found is the reason, one of the major reasons for this is [00:14:40] inflammation with the capillaries. And so as we get older, [00:14:45] we hear this term inflammaging.

And so this inflammaging builds up [00:14:50] inside of our capillaries, and it actually pinches the capillary shut so the red blood cells can't get [00:14:55] through. And all of your cells downstream go from aerobic respiration [00:15:00] to anaerobic respiration where they're using glucose in the absence of oxygen to [00:15:05] produce energy.

Now. There's a problem with that. If you look at [00:15:10] compare aerobic respiration to anaerobic respiration, aerobic respiration [00:15:15] produces about 16 times as much energy. So now you've got this [00:15:20] cell that's kind of on life support. It has to cut out non-essential [00:15:25] functions. Things like taking out the trash.

It just lets all this metabolic waste build up inside the cell [00:15:30] walls. It, it's at a low energy state. What he found was [00:15:35] that by doing ey, a couple things happen. First, we start saturating our [00:15:40] blood plasma with oxygen. And now the way our blood [00:15:45] works is we got these red and white blood cells that float in this clearish brown liquid called blood plasma.[00:15:50]

The red blood cells normally carry like 98% of our oxygen, [00:15:55] and the blood plasma has very little. But when you're doing wo because of the massive amount of [00:16:00] oxygen you're bringing in, you actually start to saturate this blood plasma. And while the [00:16:05] red blood cells can't get through, the blood plasma is much thinner and it can get [00:16:10] through there and start oxygenating the cells downstream.

Things that have been choked [00:16:15] off from oxygen for perhaps a long time. And then the second thing it does is it [00:16:20] creates an anti-inflammatory effect and it actually causes the linings of those [00:16:25] capillaries to to reestablish normal blood flow. And so he [00:16:30] had some elderly folks do. A few sessions of E wat and then he had 'em come back like [00:16:35] three months later and he, he measured 'em and he saw that there was a durable change.[00:16:40]

It wasn't just opening these capillaries while they were doing e wat, it continued to remain [00:16:45] open and it, it reestablished more youthful blood flow. And so that's [00:16:50] more youthful oxygenation of your cells. It reignites, [00:16:55] mitochondrial biogenesis and ramps it up. So that is the root of [00:17:00] it. One of the most powerful thing, it does so many things.

So [00:17:05] oxygen is intensely anti-inflammatory. So there's a lot of [00:17:10] research out there that shows anywhere in your body [00:17:15] you have low oxygen, you have inflammation. Anywhere you have [00:17:20] inflammation, you have low oxygen, you can't have one without the other. So obviously when you start flooding [00:17:25] your tissues with oxygen, you're creating this powerful anti-inflammatory effect, [00:17:30] not just in your capillaries but everywhere.

And so, as I'm sure all your listeners [00:17:35] know, inflammation is at the root of basically every chronic health condition we have out there. So [00:17:40] that's the first thing. It's anti-aging, it's anti-inflammatory. Obviously it's important [00:17:45] for energy. It increases our energy production, and that's not just about us feeling like clearheaded and, and [00:17:50] having energy for the day.

That's about your liver, your brain, your skin, all of them doing the [00:17:55] things they need to do at their best. And then. The last thing [00:18:00] really I, I think is really potent about it is it's power for [00:18:05] detoxification. The one thing I don't hear many people talk about, which surprises me, is actually 70% of your [00:18:10] body's detoxification actually happens through your lungs.

That's your biggest detox organ. And [00:18:15] so obviously breathing in and out, oxygen is helping there, but oxygen helps [00:18:20] detox in a variety of ways. We talked about. Your body being in a low energy state and building up [00:18:25] all these toxins inside the cell wall. Well, when you flood them with oxygen and they [00:18:30] reenergize, they can start dumping all these toxins out.

Once they, once they get out into [00:18:35] the system, you're doing exercise, which is really the power of eod. It's that [00:18:40] synergy of massive amounts of oxygen and all of the benefits of exercise [00:18:45] and. So what happens is now you're moving your lymphatic system [00:18:50] around, you're moving your lymph fluid, so you're moving that stuff around.

And then lastly, when you have [00:18:55] toxins in your system, you need to oxidize them, which is you use oxygen in a [00:19:00] reaction to break them down into simpler substances that allow your excretory [00:19:05] organs to, to move them outta your body. We're doing all of these things [00:19:10] simultaneously. And so that's really the power is anti-inflammation, detoxification, more [00:19:15] energy, uh, all at once when

Dave: people are using your LEO [00:19:20] system.

What does it look like? Like you, you put a mask on, you're on a treadmill. [00:19:25] How long do you go? Like what, what does the protocol look like?

Brad: Yeah, and it's, it's really relatively simple. [00:19:30] So the equipment looks like an oxygen concentrator is this device that's probably a foot and a [00:19:35] half tall, and it's maybe a foot and a half wide by a, a foot thick, if you will, [00:19:40] and you plug it into the wall and it's got oxygen tubing coming out.

The challenge with [00:19:45] these devices is they don't produce nearly enough oxygen for you. So we, the two systems I [00:19:50] talked about, one produces 10 liters a minute, and the other one produces five liters of oxygen a [00:19:55] minute. When you're exercising, you can breathe in like 50 or [00:20:00] 60 liters of oxygen a minute, quite easily.

And so what we do is we use, [00:20:05] rather than try to ha gang up 10 of these machines on you and, and you know, watch your, your power [00:20:10] grid fail. What we do is we have one machine and we use it to fill a large reservoir. [00:20:15] So this, in our case, it's a, it's a, a what looks like a bag, which is like six feet [00:20:20] wide and five and a half feet tall.

Our, our reservoir can hold a thousand liters. [00:20:25] So if you think about a two liter bottle of soda, you know it's 500 of those. So it's a lot of oxygen. [00:20:30] So we fill that reservoir up. And then from there there's a hose that comes to a [00:20:35] mask that you wear and. You, basically just you. That's what the equipment [00:20:40] looks like as far as the protocol.

You put that mask on and you do 15 minutes of [00:20:45] cardiovascular exercise with the mask on, and you're breathing 93% pure [00:20:50] oxygen. We have it designed with a specific. Two-way valve. So one [00:20:55] valve opens when you're breathing in pure oxygen, and then when you breathe out it shuts. So none of [00:21:00] your exhale gases go back in the reservoir.

They, they exit out a separate, uh, [00:21:05] valve into the room. And in that way the next breath is also 93%. And so we're keeping it a [00:21:10] high percentage of oxygen for that period of time. Now that people [00:21:15] can easily use 900 plus liters of that reservoir in a 15 minute [00:21:20] session. And. To give you a feel for how much oxygen that is.[00:21:25]

That's about the same amount of oxygen a person uses at rest in like 14 hours. You said you've [00:21:30] got soft shell and hard shell. It's comparable to spending an hour in a hard shell chamber sort of thing. [00:21:35] So it's a massive amount of oxygen and we're able to do it quickly because of the [00:21:40] synergies of exercise and all the.

Physiological changes that occur in our [00:21:45] body when we're exercising.

Dave: So you get this surge of oxygen and you said 15 minutes. [00:21:50] Is this like zone two? Are you doing high intensity intervals? Like what's the protocol you like? [00:21:55]

Brad: So for me, uh, what I, we deal with a lot of folks with chronic illness and [00:22:00] some of them it can only start with like walking or stretching or, and some of [00:22:05] them, if they're very sick, oftentimes.

What can happen at the very beginning, [00:22:10] particularly for sick people, is they can have a detoxification reaction. All of a sudden, your cells start dumping [00:22:15] all this toxic waste. It overwhelms your elimination organs. So for folks who are [00:22:20] very sick, we kind of say, you kind of, you kind of walk into this, so to speak, no pun intended, but you kind of [00:22:25] start slow and build up to that 15 minutes.

And, but the end state kind of intensity we like for [00:22:30] folks is about 70 to 80% of your theoretical max heart rate. And you wanna try to hit [00:22:35] that for that 15 minutes, that's pretty hard to do for most people. Yes. Well, and that's the other thing [00:22:40] we do tell folks is we say. You know, perfection is the enemy of progress in this sort of [00:22:45] thing.

So if you're doing it, you're gonna get benefits. Go out and do your best. The thing [00:22:50] is, when you're exercising with oxygen, it's much easier. And you, [00:22:55] so, and I always give this example, like when I started doing it. And you, you can attest to this, [00:23:00] Dave, is I would do my elliptical at a, at a resistance of 12, which was pretty [00:23:05] stiff at the time.

And then I threw that mask on and immediately I had to go up to 13 and 14. [00:23:10] And then a month later I had to go up to 15. And today, you know, I do it at like a 17 and it doesn't feel [00:23:15] any different from when I used to do it at a 12. So your endurance improves and your, [00:23:20] your ability to perform exercise improves dramatically.

As a matter of fact, a [00:23:25] lot of the research on Wot was done by, by, you know, the, the US Olympic [00:23:30] team and some Russian Olympic teams. So there's a ton of research on it, improving athletic import [00:23:35] performance.

Dave: It's funny, a lot of the biohacks came out of Russia and East Germany during the [00:23:40] Olympics in the, in the eighties.

They were just willing to do all the crazy stuff and, you know, some of the [00:23:45] fundamental things like whole body vibration came outta that. And then nasa, some of the stuff around lasers and [00:23:50] certainly this oxygen stuff. So there's a lot of research done early on on [00:23:55] it. And it's uh, it's basically a 15 minute workout.

And I've seen people do it with [00:24:00] high intensity interval training. And I've also seen people do, you know, two [00:24:05] minutes on, one minute off and all kinds of different protocols. How do you know which ones are the best ones? It really

Brad: [00:24:10] depends on your goal. Okay. So a lot of, uh, I start from a place of people with [00:24:15] chronic illness 'cause I have a lot of empathy for that.

If you're dealing with chronic illness. Uh, none, [00:24:20] none of this hit stuff. I mean, we don't want you to wear your body out. We're trying to heal your body from the inside [00:24:25] out. So it's, this isn't about killing it, this is about getting your heart rate up for an [00:24:30] extended period of time. What we're really trying to do with ey, like I said, it's about the physiological [00:24:35] changes.

When, when you start exercising, your body immediately recognizes it [00:24:40] does not have enough oxygen to produce the energy to keep the exercise up. [00:24:45] Compensates, you start breathing heavier, deeper more frequently your heart starts [00:24:50] beating deeper and more frequently you have vasodilation. All these [00:24:55] compensatory things happen to allow your body to carry more oxygen, and we are effectively.[00:25:00]

Hijacking, all of those changes to bring a massive amount of oxygen into your [00:25:05] body in a short period of time. So what you wanna do is you wanna get your heart rate up [00:25:10] and you wanna get your blood pumping. And that's really what we're going for. And then, and it's not about [00:25:15] perfection, it's about progress.

So that's for on the chronic illness side. Now, [00:25:20] if you're a high performer looking to get the best out of it, there's two great [00:25:25] ways to use EY or two great benefits, and there's kind of a little bit of a different use for each. [00:25:30] One is it'll improve your vo O2 max. It'll improve your [00:25:35] endurance, it'll improve the intensity at which you can work out.

So you're, what you want to [00:25:40] do is you want to, you wanna do OT when you're doing some of those intensity sessions. So I mentioned [00:25:45] myself. I went from a resistance 12 to resistance 17. Now there's days I, [00:25:50] I don't fill the reservoir and I, I go out to go do my workout. And it's not filled, and I still [00:25:55] go do it at 17.

So it's not like I need that mass to do it. I've, I've built that endurance. [00:26:00] So you wanna do it on those, those push days and, and really help push yourself. 'cause you're gonna be able [00:26:05] to do more work and train your muscles to do more and get your system going and more [00:26:10] mitochondrial, biogenesis and all that.

The second way, equally as [00:26:15] powerful for people who are looking for high performance is to use it for recovery. [00:26:20] So when your body runs out of oxygen and it switches to anaerobic respiration, it produces a [00:26:25] massive amount of waste products including lactic acid, which is what makes our muscles sore. When [00:26:30] we use, when we use oxygen.

It actually, you need oxygen to break down [00:26:35] lactic acid, and so when you do it for recovery, it helps you so you can get to [00:26:40] the next thing more. More quickly. And my example there is like [00:26:45] weightlifting, I'll wi I'll lift weights a couple times a week and I'll do wot right [00:26:50] afterwards, but if I miss a week or two because I'm traveling or something's come up, [00:26:55] I, I never get muscle soreness, which is like shocking to me.

'cause when I was like 33, [00:27:00] if I missed a week and a half and I tried to come back, that hurt. So it's, it's kind of the [00:27:05] magic of oxygen. So. It, it's those two places where you wanna use it. You wanna use it to push, to [00:27:10] improve your performance, and you wanna use it for recovery to help get yourself [00:27:15] back for that next session sooner.

Dave: How do people feel when they do it? Some of the targets for biohacking, things [00:27:20] like brain fog energy levels, VO two max, HRV. How, how would I know what's [00:27:25] working?

Brad: You know, the interesting thing is we have a lot of folks, you know, obviously we have a lot of folks who are [00:27:30] biohacking and doing it, and many of 'em come back to us and just say, this is my favorite thing.

'cause it's the [00:27:35] most noticeable results. We have folks who get on and immediately they noticed a, a, a [00:27:40] boost in energy, in clear headedness, uh, and, and in sleep quality. [00:27:45] Those things they'll notice right away or they'll notice they're able to do more exercise than they've been able to do [00:27:50] before. Some people don't notice that I was actually in the former camp where, or in the [00:27:55] latter camp, excuse me, where I didn't notice any benefits from OP because I was so sick [00:28:00] that it took about three months before I started to notice the benefits.

Because I had so much in, [00:28:05] I was described like a bathtub filled with inflammation. I was draining it. And you just don't notice it's [00:28:10] draining until it kind of comes down to a level. You're like, wait a second. Wasn't this thing to the top, it's now eight inches [00:28:15] down sort of thing. So it kind of depends.

Most folks will notice [00:28:20] something within a few weeks. You'll notice clear headedness, you'll notice energy, you'll notice [00:28:25] sleep, and you'll notice better athletic performance. That's, uh, that's pretty noticeable. The [00:28:30] day of it though. You ought to feel like clearing of brain

Dave: fog, right?

Brad: Yes. A lot. A lot of folks, [00:28:35] myself included, I've noticed that where clears your brain fog.

So brain fog [00:28:40] really comes down to a lot of times inflammation in the brain. Our brain uses like. [00:28:45] 20% of our oxygen and it's only 2% of our mass. So it's the biggest oxygen consumer. [00:28:50] So when you're flooding your body with oxygen, obviously it's getting a ton of [00:28:55] oxygen. It's relieving that inflammation and it's helping your, all of your cells go back to [00:29:00] aerobic and be able to do everything they need to do.

So we do have folks who notice a lot of clear headedness. [00:29:05] I've often joked that this should be. We should put these on college campuses for the [00:29:10] kids the day after 'cause just clear their head out and clear, uh, all of the, uh, sins of yesterday out. [00:29:15] Yeah, for hangovers,

Dave: I like it. It seems like it would pair well with something like Vaskin for my friends at [00:29:20] Roy.

Supplements that open up your capillaries so you get better blood [00:29:25] flow or anything with nattokinase in it. Uh, STEM regen has something [00:29:30] like that. I think it's their mobilized product and. The idea here is more [00:29:35] liquid blood or more open capillaries would make a huge difference. While you're doing this, [00:29:40] I would want to.

Probably pre-treat if I was gonna do it. What else should people [00:29:45] do before or after they do a session on ot?

Brad: Yeah, those are [00:29:50] great ones. Those in LA nine. Anything that leads to vasodilation is, um, [00:29:55] is, is a, is an awesome idea to kind of boost the power of it. As far as [00:30:00] afterwards, there's not a whole lot you necessarily need to do, but what I do love [00:30:05] is I do love to pair it with red light, and there's a couple different ways you use it, depending on what you're [00:30:10] using it for.

So if we think, the way I think about it is. Now red [00:30:15] light basically works from an oxygen standpoint as well. A lot of folks don't [00:30:20] realize this, but OTTs all about driving oxygen supply. It's a massive amount of supply. It's [00:30:25] 14 hours worth of oxygen in 15 minutes, flooding your system, and then [00:30:30] red light kind of works on the inverse.

It's all about oxygen demand. It stimulates [00:30:35] your mitochondria to suck in oxygen and produce a TP to generate more [00:30:40] energy, to do everything. They do their best and while. TTS coming from the inside out, [00:30:45] deep within. The great thing about red light and near it for red light is it's [00:30:50] from the outside in.

And so we're hitting, if you will, that last mile delivery, so to [00:30:55] speak. And so it pairs really well with that. And there's two ways you use it. For, [00:31:00] if you're going for athletic performance I recommend you do the red light first. [00:31:05] So the reason is now all of a sudden your your mitochondria are fired up and they're ready to suck [00:31:10] in oxygen.

And now you're, you're throwing in a huge amount of supply and you're gonna [00:31:15] improve the amount of, uh, work you can do, the amount of exercise you can do in the [00:31:20] performance for people with. Chronic health conditions, it's a little bit of the opposite. I, [00:31:25] I recommend they do it afterwards. So you first flood your body with oxygen, [00:31:30] so that has all the supply.

When you get off the exercise equipment and you go lay down underneath [00:31:35] that red light or, or however you're using it, you've got that excess supply still floating [00:31:40] around and then you turn those red lights on and and generate the demand, which is pulling more of it in and it's already [00:31:45] there within your system.

Because for those folks asking for more demand when they [00:31:50] have kind of a supply problem due to inflammation, is a challenge. So that's how we stack that.

Dave: [00:31:55] Okay.

Brad: I like it.[00:32:00]

Dave: It seems like something like coq 10 might be really helpful. Anything that helps mitochondria, Mito. [00:32:05] Uh, MIT oq, uh, PQQ timeline. Like you, you should take a full [00:32:10] mitochondrial stack at the same time, shouldn't you?

Brad: You can definitely do that. What, um, [00:32:15] and all of those things can help speed up the end game is what it really comes down to, because.[00:32:20]

At the end of the day, you are supplying so much oxygen that you, [00:32:25] you generate this mitochondrial biogenesis at such a large level that it, [00:32:30] you kind of effectively, over time, you optimize. Now sometimes over [00:32:35] time, like for me. I continued to see incremental benefits from EY into my third [00:32:40] year, which I found is pretty shocking.

You know, any, you think any kind of exercise [00:32:45] program you're doing, you're gonna like level off at like six months or something. [00:32:50] Now doing something like that will probably compress the timeline. You probably won't be looking at three years. You can [00:32:55] probably bring it down to, you know, 20 months or, or two years sort of thing By something like that, what.[00:33:00]

Aging markers change when you're doing OTT

Dave: therapy.

Brad: Oh, you [00:33:05] know, obviously inflammation comes way down, which is a big one. Energy [00:33:10] production goes way up. I had, so this same, you know, [00:33:15] I'll tell my story on this one and I don't remember, I don't remember all of the age markers on this, but so [00:33:20] that same doctor who recommended OT a few years later, I went in and he says [00:33:25] maybe a year or two later, I still wasn't all the way better, but I was on the way.

And he said to [00:33:30] me, Hey, you know, I've got this new machine. It's called a bioenergy tester, and I [00:33:35] wanna have you do it. I wanna get you in my database to see what you can do. He just had a hunch. [00:33:40] So basically this was a machine where they had you, you know, basically you sat down and took your blood pressure, your heart [00:33:45] rate, your breathing rate, all these biomarkers.

And then while you're [00:33:50] resting, first thing in the morning for me, and then they put me on the bike and the nurse says, look, this bike is [00:33:55] gonna keep her aggressively increasing intensity until you fail. And fail is I think [00:34:00] your heart rate gets up to some point or. Your breathing rate, and she said, it'll take you about [00:34:05] eight to 12 minutes.

And so I said, okay. So I got on the bike and I just kept [00:34:10] going and it was eight minutes, no, 12 minutes, no big deal, 20 minutes, no big deal. Finally at like [00:34:15] 24, 25 minutes or something like that, finally it hit. And so I got off the bike and she was just [00:34:20] shocked, staring at me. She's like, what are you some sort of ultra endurance athlete?

I laughed, I'm like, [00:34:25] no, I'm, I'm definitely not doing that. I'm like, just trying to get over this chronic illness. And they. [00:34:30] Gave me all these test results and they looked at my mitochondrial [00:34:35] function. They looked at they looked at my, my, my heart health and, and you know, some of my [00:34:40] other organ health sort of thing.

And they're like, they were telling me they were quoting me that I was 10 [00:34:45] and 20 years younger than what they would've expected for someone my age. It was, it was really kind of [00:34:50] shocking. And at that time I still wasn't like a hundred percent. So it was, it was kind of shocking to [00:34:55] me. Incredible. One

Dave: of the guys who did talk about EWA and may have been my inspiration was Frank [00:35:00] Shallenberger, um, back in the day.

And he's been a major proponent of [00:35:05] ozone therapy as well. And. So it's, it's all about [00:35:10] mitochondrial function here. But other than that research, I haven't seen much in the literature [00:35:15] about OT unless I just missed it. So have you done new studies? Are there other people doing studies? [00:35:20]

Brad: There are new studies.

There's there was some studies at a University of Cincinnati on [00:35:25] COVID that they were working on long COVI. There's actually was one in [00:35:30] 2018 on mitochondrial biogenesis. Um, the, [00:35:35] if you look up, so the term you research, if you're looking to look up ewa, it's called [00:35:40] Hyperoxic, uh, you know, hyperoxia or Hyperoxic exercise.

So [00:35:45] hyperoxic is just the fancy medical term for high oxygen. It's the opposite of [00:35:50] hypoxia, which is low oxygen. So hyperoxic exercise, there's a ton on that. [00:35:55] There's, so obviously Von Ardine did a, a lot of great things [00:36:00] on, like the cancer. He showed that he could u by using, he could take a, a, a cancer [00:36:05] cell and unpr it, so to speak.

Um, and he did some of the anti-aging research, which is [00:36:10] foundational. We have, since then we've had a lot of sports, um, and [00:36:15] athletics, but. The other thing that it's, it's relevant to it is anything you see [00:36:20] with, um, hyperbaric is generally relevant to EWA because there are two [00:36:25] modalities that are effectively using high volumes of oxygen to [00:36:30] flood the blood plasma and the body with oxygen.

So they're pretty [00:36:35] much, oftentimes they're analogous in that respect.

Dave: I, I can say there's a huge [00:36:40] amount of new research. Uh, we just did an episode, um, looking at the different pressures of [00:36:45] hyperbaric oxygen that 1.3 atmospheres does something different. And I feel like in the world of biohacking, [00:36:50] this idea of manipulating, um, the mix of gases and the [00:36:55] pressure of gases, um, is just now coming into maybe it's [00:37:00] teenage years in terms of being known.

And we definitely use a hyperoxic [00:37:05] hypoxic system at Upgrade labs. I mentioned, you know, I have the Oxy health system [00:37:10] downstairs, uh, and it's definitely helped a lot with brain injury and [00:37:15] I've, you know, first used e watts. Good god, I'm dating myself. That is, [00:37:20] is that really 31 years ago? It, it is. Geez. [00:37:25] so all of these are part of my biohacking stack and I feel like over the next few [00:37:30] years, um, way more is gonna come out.

In fact, I'm waiting delivery of a xenon gas [00:37:35] cylinder. I wrote about that in, that was in my longevity book in superhuman, [00:37:40] uh, where it actually increases EPO and gets more blood cells going without having to use [00:37:45] EPO. So I don't think the world Anti-Doping [00:37:50] Association would complain, but if they would, that just mean it works better.

So my, my [00:37:55] personal mission is to remove the anti-doping association from the planet because it's a, it's a [00:38:00] scam and they're harming athletes and I like athletes. So, [00:38:05] um, okay. So what else is gonna happen in the future of uwa? So you've got a, [00:38:10] a very accessible system. I would say it's the best one on the market.

Also very [00:38:15] fairly priced. Um, and it, and the bag you used is like, it, it, it's a [00:38:20] good bag. The original stuff I'd tried, um, from the $10,000 guys, it was [00:38:25] just, it was not a good experience. We'll put it that way. Um, so, what [00:38:30] you've got. Works pretty well. It takes up some space in a room, but you put it behind a couch or something, it's [00:38:35] probably not gonna, you know, win any decor competitions except for maybe Halloween.

'cause you [00:38:40] put like big googly eyes on it. But it's like, it's, it's totally functional for someone [00:38:45] who's recovering, um, or for a pro athlete. Um, but what's coming next? Are you gonna, you [00:38:50] know, attach oxygen sensors add some kind of switching, like, like where's it going? [00:38:55]

Brad: So we don't generally do the switching that you referred to the altitude switching [00:39:00] where people, you, you don't now, but I'm saying like, where's the future going?

Yeah, we, we haven't [00:39:05] had interest in that. And I'll tell you why. Because our real goal is about [00:39:10] serving folks who have chronic illness and helping them get better. And the issue with the low [00:39:15] oxygen switching, so there's one company that was really big into that and there was a couple other [00:39:20] companies that did it, and.

Several of 'em kind of had gotten outta that. But the issue [00:39:25] is any, like I had mentioned earlier, anywhere in the body you have low oxygen, you have [00:39:30] inflammation. And so a lot of our customers are dealing with runaway inflammation. And so [00:39:35] we don't recommend that, quite honestly, if you want to do that, we tell folks the easier way to do [00:39:40] it is you can do breath holds while you're exercising.

'cause a breath hold is 0% oxygen. It's even more [00:39:45] severe than low oxygen. It doesn't require you trying to turn a switch while [00:39:50] you're trying to balance on a machine sort of thing. But for people with chronic health, we don't do [00:39:55] that. Our goal is to build out a suite of available [00:40:00] products for folks with chronic health conditions and make them affordable things that are science [00:40:05] backed.

You know, you know this Dave, you learn so much getting better, right? Like [00:40:10] nowadays I see so much of it's actually becoming mainstream, more and more of it. But you know, there's so [00:40:15] much I've learned too and I wanna bring some of that out and bring that to folks a and bring more of that. [00:40:20] So there's some other ideas we have in the can, even beyond wot.

But as far as the, you know, the [00:40:25] future of EY I think is less about the technology upgrades to [00:40:30] it because like, you can't. You can't measure the oxygen difference. So you [00:40:35] can't put one of those pulse oximeters on your fingers No. And see a difference because you're at

Dave: 99%, [00:40:40] you're at 99%, right? Yeah.

Brad: It's not, it's measuring red blood cells and what we're talking about plasma, which you [00:40:45] can't measure, right?

So there's not like a feedback mechanism that's readily available to the [00:40:50] public. Really. I think it's really about a revolution in availability, right? [00:40:55] If we can get more people to make it available, that's the big revolution. I mean. In the OTT [00:41:00] space for me.

Dave: So just getting it out there, making it more affordable.

Any [00:41:05] opportunities to make it more portable? More

Brad: portable? That's a, that's a good one. I [00:41:10] wish there were, the challenge is if you need 60 liters of a gas every [00:41:15] minute you're exercising, imagine trying to run outside while you're carrying a, a floating balloon [00:41:20] with, uh, you know, you're gonna hit the power lines and, uh, you know, I don't know if I've got the insurance for that.

[00:41:25] So, uh, just to be honest, you so. Then you're talking about compressed [00:41:30] gas. And compressed gas is where you get into a dangerous situation. You don't wanna have compressed [00:41:35] oxygen if you have a leak or something that gets into a challenging situation, so that's difficult. [00:41:40] There are ways to make instantaneous generations, so you don't need that reservoir.

[00:41:45] Unfortunately, those are price prohibitive for home use. Now, if you're talking about that for clinic use, I think that's [00:41:50] something that you're going to see more and more of in the future.

Dave: And for highly motivated [00:41:55] listeners, you, you can buy probably four oxygen concentrators and put 'em all [00:42:00] together and get enough to be useful, right?

Brad: Probably not Four, probably five or six. Five. Okay. [00:42:05] You could now. Here's the challenge with that. The challenge with that [00:42:10] is, as I mentioned, our mask and what we use with these kind of traditional EW watt systems, it seals [00:42:15] around your face and it has this two-way non-rebreather valve so that you're not breathing back in your exhale [00:42:20] gases if you're trying to gang up these concentrators.

Using a, an oxygen [00:42:25] tubing, it's gonna be forcing, like you're gonna feel like you're underneath a pressure washer of [00:42:30] oxygen, and likely you're gonna have a mask that leaks around the edge. So you're not gonna get quite as [00:42:35] good of a workout as if you had a sealed mask in a, in a kind of a, a, a, a bigger, um, [00:42:40] hose feeding you.

Okay. That's a fair point.

Dave: Do people mix it with with ozone? Like do you do [00:42:45] ozone before or after? Do you mix it? Maybe do hyperbaric one day, eat water another [00:42:50] day? Like what

Brad: do we know what works? You can mix it with ozone, no problem. I've mixed it with ozone. I've used it [00:42:55] for, um, like I've said, I've, I've used ozone for a variety of things and no issues with that [00:43:00] whatsoever.

They work in a little bit of different. Ways and, and kind of, I think of them as [00:43:05] different purposes, although obviously you do get oxygenation through ozone as well. [00:43:10] It's also the, um, it's also basically causing all the free radical [00:43:15] changes in the body, I believe that have a chain reaction. As far as with hyperbaric, you can, like I said, [00:43:20] they're effectively offering similar benefits.

We do. [00:43:25] Generally see folks tell us, and you, you can tell us if this is true for you, Dave, it's a more [00:43:30] noticeable with OTT versus hyperbaric early on, they notice a, a bigger [00:43:35] difference when they start doing it. But we do have some folks out there who, we have folks out there who've [00:43:40] come from hyperbaric and go to ott.

We have some folks out there that are doing a little bit of each as well. [00:43:45] So you can mix 'em. There's no harm in it whatsoever. As long, you know, with [00:43:50] inbounds obviously like. There are folks out there who are doing alternate cancer protocols who are doing, [00:43:55] you know, and there's people out there recommending you do OTT three times a day.

I don't, there's [00:44:00] no harm in doing that. I just think you're, you're, you're overworking your body. 'cause there's exercise involved sort of thing. [00:44:05]

Dave: Oxygen is also a prooxidant at very high levels. You can get an [00:44:10] overdose of oxygen. I'm, I don't know that I would recommend three times a day. And, and we know if.[00:44:15]

Hyperbaric that the pressure change, that going up and going down appears to have a pizo [00:44:20] electric or some other effect. So I would say it's okay to mix 'em. [00:44:25] Uh, and it's okay to say, well, I wanted to do e watt because it was [00:44:30] abundantly affordable and I could do it at home. Right. And [00:44:35] there's nothing wrong with either one of those.

So I, I think if I was saying, all right, I'm facing [00:44:40] cancer or this autoimmune stuff that we've both dealt with. I would definitely want an ozone [00:44:45] machine. I'd want your OTT machine. I'd want a soft sided hyperbaric or [00:44:50] access to that kind of stuff. Like I'd wanna live next door to an upgrade labs. Yeah, right.

Go in there and [00:44:55] get the hard shale stuff. Yeah, exactly. Um, although actually, strangely at, at [00:45:00] upgrade labs, we don't do hyperbaric for a good reason. Most people need 40 [00:45:05] sessions and then they're done and they're very long and, you know, people get claustrophobic. And also I find that. [00:45:10] Leasing a soft shell for, for use like that.

People do it, they do their whole [00:45:15] family, they save a ton of money, and then they're done. And it takes a lot of space. So it's a, it's rough to run a business that [00:45:20] does hyperbaric and other stuff, um, but to, to, uh, do something instead. We do [00:45:25] hyperoxic hypoxic training, which is in the same universe while we're talking about.

It's just more, more time and, [00:45:30] uh, and cost efficient for, for our clients. So the, [00:45:35] the general lesson for our episode. Is that [00:45:40] increasing metabolic demand in a very high oxygen environment has unique [00:45:45] metabolic benefits that turn off in some cases, tons of autoimmune [00:45:50] inflammation. My view on things has just only gotten stronger over the years that [00:45:55] autoimmune issues are ultimately mitochondrial.

You fix the mitochondria, [00:46:00] drop the stress response, the cell danger response, and you fix everything. So having abundant [00:46:05] energy in the cells and working cell membranes and working redox potential seems to [00:46:10] fix almost everything. I don't care if you're talking about Parkinson's or Alzheimer's or cancer, like the, those all [00:46:15] could be symptoms of mitochondrial dysfunction.

Now, the cause of the mitochondrial dysfunction could be, [00:46:20] you know, in just Alzheimer's, uh, Dr. Dale Breen came on the show. It [00:46:25] is like, oh, there's seven kinds of Alzheimer's. Right. Okay. Or, um, even things like a [00:46:30] DHD, uh, or autism. Daniel Amen on his board of directors, a good friend been on the [00:46:35] show multiple times saying, well, there's different kinds of A DHD, but all of them are symptoms.

Mitochondria, [00:46:40] mitochondria, mitochondria. What you did, and what I did was reboot and enhance mitochondrial [00:46:45] function, which resolves all sorts of weird stuff.

Brad: Yeah. You know, one of my favorite [00:46:50] quotes is from, uh, that right on that strain is from Arthur Geen. So [00:46:55] Arthur Geen wrote. This medical textbook called Medical Physiology in the [00:47:00] 1950s.

And every doctor, any one of us or anyone listening to the show has ever gone and [00:47:05] seen, had to use this medical textbook. It's still the gold standard today. So [00:47:10] very well respected guy. And he said and I'm paraphrasing here, but [00:47:15] basically the root of all disease is lack of oxygen. To your point, that becomes a metabolic [00:47:20] issue because, at that point that when our metabolism goes from aerobic to anaerobic, then we don't [00:47:25] have, we don't have the energy we need. Oh,

Dave: makes so much sense. In fact, if you [00:47:30] go back to maybe the first 10 episodes, this was like 13, 14 years ago Dom [00:47:35] dagostino came on talking about hyperbaric oxygen with ketones, uh, [00:47:40] as a form of cancer treatment.

So like, gee, do ketones cause [00:47:45] more electrical output for mitochondria? Yes. That leads me to a question. It [00:47:50] seems like I'd want to be in ketosis when I'm doing OT with your system, right?

Brad: Yeah,

Dave: yeah, absolutely. [00:47:55] That's one of those things where all these biohacks that, you know, we've been talking about for a decade, they all [00:48:00] stack and they're all ultimately, let's get more blood there.

Let's get more oxygen there. Let's get more mitochondria working. [00:48:05] Let's get the right signal into the body and the signal that you're putting in with [00:48:10] your system, Brad, is. High metabolic demand and high oxygen. Therefore, it's safe to [00:48:15] increase your capacity and then when the demand goes away, the capacity remains increased and then you improve.

[00:48:20] So it would make sense if that works for exercise, uh, for professional exercisers or athletes. [00:48:25] Um, and it works for chronic illness, and I very much appreciate that you've. lowered [00:48:30] the price to make it accessible, uh, without democratizing E wat. The reason [00:48:35] is that I don't think other people should be allowed to vote whether or not you're allowed to do e watts.

So we don't need democracy for [00:48:40] this. We just need affordable pricing. I agree with you. [00:48:45] Less regulation, more freedom. Right. Lower costs, amazing levels go together. If I can

Brad: [00:48:50] keep them outta my healthcare, that'd be great.

Dave: Thank you. Exactly. Exactly. Well, I, I very much [00:48:55] appreciate the work you're doing, guys, is 1000 roads.com [00:49:00] and if you're thinking about some kind of oxygen therapy, this is a very affordable one [00:49:05] and it's something that can make a difference.

Downside, you're gonna have, uh, [00:49:10] a balloon in your living room. Uh, or wherever you put the thing. It's less [00:49:15] of a footprint than I think you'd have with hyperbaric, which is also a big balloon in your living room. So, [00:49:20] uh, if you're into balloons, oxygen therapy may be your thing. If you are not into balloons, you could get an [00:49:25] ozone device, which isn't about the same price point, um, that you just have to.

[00:49:30] Be okay. You hate balloons, but you don't mind squeezing a balloon to put ozone in your butt. So [00:49:35] really this is a balloon centric episode now that I think about it.[00:49:40]

On that note, Brad, thanks again. Hey, thank you so much, Dave. You have a great one. [00:49:45] See you next time on the Human Upgrade [00:49:50] Podcast.
"""

In [6]:
# Pydantic Models and Setup. Pydantic Models will be merged with Mongo Beanie Models 
# Through some class composition  
from pydantic import BaseModel, Field  
import re   
from typing import Optional, List, Dict, Any, Literal  


def clean_whitespace(text: str) -> str:
    return re.sub(r"\s+", " ", text).strip()  

transcript_text = clean_whitespace(transcript_text)


def extract_attributions(summary_text: str) -> str:
    matches = re.findall(r"<attribution>(.*?)</attribution>", summary_text, re.DOTALL | re.IGNORECASE)
    return "\n\n".join(m.strip() for m in matches) if matches else ""


class AttributionQuote(BaseModel):
    text: str = Field(..., description="Short quote from summary/transcript")
    timestamp: Optional[str] = None
    speaker: Optional[str] = None 
    chunk_id: Optional[str] = None  


class ProductItem(BaseModel):
    name: Optional[str] = None
    description: Optional[str] = None
    features: List[str] = Field(default_factory=list)
    protocols: List[str] = Field(default_factory=list)
    benefits_as_stated: List[str] = Field(default_factory=list)
    attribution: List[AttributionQuote] = Field(default_factory=list)


class ProductSummary(BaseModel):
    products: List[ProductItem] = Field(default_factory=list)


class TreatmentItem(BaseModel):
    name: Optional[str] = None
    procedure_or_protocol: List[str] = Field(default_factory=list)
    outcomes_as_reported: List[str] = Field(default_factory=list)
    risks_or_contraindications: List[str] = Field(default_factory=list)
    confidence: Literal["high", "medium", "low"] = "medium"
    attribution: List[AttributionQuote] = Field(default_factory=list)


class MedicalSummary(BaseModel):
    treatments: List[TreatmentItem] = Field(default_factory=list)


class SectionItem(BaseModel):
    title_or_topic: Optional[str] = None
    approx_time_range: Optional[str] = Field(None, description="e.g., 00:00–05:30")


class HighLevelOverview(BaseModel):
    purpose: Optional[str] = None
    participants: List[str] = Field(default_factory=list)
    main_sections: List[SectionItem] = Field(default_factory=list)
    key_takeaways: List[str] = Field(default_factory=list)
    attribution: List[AttributionQuote] = Field(default_factory=list)


class ClaimItem(BaseModel):
    speaker: Optional[str] = None
    claim_text: str
    claim_type: Literal["causal", "quantitative", "experiential", "other"] = "other"
    evidence_present_in_transcript: Literal["yes", "no"] = "no"
    attribution: List[AttributionQuote] = Field(default_factory=list)


class ClaimsSummary(BaseModel):
    claims: List[ClaimItem] = Field(default_factory=list)


class CompanyItem(BaseModel):
    canonical_name: str
    aliases: List[str] = Field(default_factory=list) 
    description: Optional[str] = None   



class CompaniesSummary(BaseModel):
    entities: List[CompanyItem] = Field(default_factory=list)


MODEL_MAP = {
    "product_information": ProductSummary,
    "medical_treatment": MedicalSummary,
    "high_level_overview": HighLevelOverview,
    "claims_made": ClaimsSummary,
    "businesses_entities": CompaniesSummary,
}


# ---------------------------
# Summarization prompts
# ---------------------------
SUMMARY_PROMPTS: Dict[str, str] = {
    "product_information": (
        "The intention is to parse the following transcript and extract information about products. "
        "Extract and summarize all information related to products, including product names, descriptions, "
        "features, and any discussions about product performance or use. Present the summary as a list of key "
        "product details. For each product, include an <attribution> section and include the attributions exactly in <attributions> ```attributions``` </attributions> containing short quotes and timestamps "
        "directly from the transcript.\n\nTranscript:\n{transcript}"
    ),
    "medical_treatment": (
        "The intention is to parse the following transcript and extract information about medical treatments or services. "
        "Extract and summarize all content related to medical treatments or services, including treatment names, procedures, "
        "protocols, patient experiences, and any relevant outcomes. Present this summary as a concise, organized overview. "
        "For each treatment or service, include an <attribution> section with short quotes and timestamps from the transcript.\n\n"
        "Transcript:\n{transcript}"
    ),
    "high_level_overview": (
        "The intention is to parse the following transcript and provide a high-level summary. "
        "Highlight the main topics discussed, key participants, and the general flow of the conversation. "
        "The summary should capture the most important points and the overall purpose of the transcript. "
        "At the end, add an <attribution> section with 3–5 quotes and timestamps that support your summary.\n\n"
        "Transcript:\n{transcript}"
    ),
    "claims_made": (
        "The intention is to parse the following transcript and identify explicit claims. "
        "Identify and summarize all explicit claims made in the transcript. For each claim, note who made it (if possible) "
        "and briefly describe the supporting evidence or reasoning provided. Include an <attribution> section with short quotes "
        "and timestamps for each claim.\n\nTranscript:\n{transcript}"
    ),
    "businesses_entities": (
        "The intention is to parse the following transcript and extract information about businesses and entities. "
        "List all businesses, organizations, or notable entities mentioned in the transcript. For each entity, provide a brief "
        "description of their role or relevance to the discussion, as stated or implied. Include an <attribution> section with "
        "a short quote and timestamp for each entity.\n\nTranscript:\n{transcript}"
    ),
}


# ---------------------------
# Structured extraction prompts
# ---------------------------
STRUCTURED_EXTRACTOR_PROMPTS: Dict[str, str] = {
    "product_information": (
        "You will extract structured product data from a targeted summary.\n"
        "Use ONLY the content in the provided summary—do not add outside facts.\n"
        "Populate the ProductSummary schema fields faithfully. If data is missing, leave fields empty.\n"
        "If the summary includes quotes/timestamps (e.g., inside <attribution>), map them into the structured 'attribution' list.\n\n"
        "TARGETED SUMMARY:\n{summary}\n"
    ),
    "medical_treatment": (
        "You will extract structured medical treatment/service data from a targeted summary.\n"
        "Use ONLY the content in the provided summary—no outside facts.\n"
        "Populate the MedicalSummary schema, set 'confidence' (high/medium/low) based on how explicit the summary is, "
        "and map quotes/timestamps into the 'attribution' list.\n\n"
        "TARGETED SUMMARY:\n{summary}\n"
    ),
    "high_level_overview": (
        "You will extract a high-level overview from a targeted summary.\n"
        "Use ONLY the content in the provided summary. Populate the HighLevelOverview schema: purpose, participants, main_sections "
        "(with approx time ranges if present), and 5–7 key_takeaways when available. Map quotes/timestamps into 'attribution'.\n\n"
        "TARGETED SUMMARY:\n{summary}\n"
    ),
    "claims_made": (
        "You will extract explicit claims from a targeted summary.\n"
        "Use ONLY the summary—no outside facts. Populate the ClaimsSummary schema. For each claim, set speaker (if given), "
        "claim_text (≤20 words if possible), claim_type (causal/quantitative/experiential/other), "
        "evidence_present_in_transcript ('yes' or 'no'), and add quotes/timestamps to 'attribution'.\n\n"
        "TARGETED SUMMARY:\n{summary}\n"
    ),
    "businesses_entities": (
        "You will extract businesses/organizations/entities from a targeted summary.\n"
        "Use ONLY the summary—no outside facts. Populate the CompaniesSummary schema with canonical_name, aliases (if any), "
        "role_or_relevance (≤12 words), first_timestamp if present, and quotes/timestamps in 'attribution'.\n\n"
        "TARGETED SUMMARY:\n{summary}\n"
    ),
} 


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  
from langchain_core.documents import Document    
from langchain_core.tools import Tool    
from langchain_core.output_parsers import StrOutputParser  
from langchain_core.prompts import PromptTemplate    
from langchain_core.runnables import Runnable    
import json 

class TranscriptIngestionState(TypedDict, total=False):
    input: str
    aggregate_summary: str
    final_summary: str   
    summary_prompts_dict: Dict[str, str]
    full_transcript: Document  
    product_information: str
    medical_treatment: str 
    structured_tool_call_dict: Dict[str, Any] 
    high_level_overview: str
    claims_made: str
    businesses_entities: str 
    structured_output_prompts_dict: Dict[str, str]
    structured_output_dict: Dict[str, Any] 
    structured_output_tool_dict: Dict[str, Tool]   
    anthropic_vertex_llm: ChatAnthropicVertex   
    google_llm: ChatGoogleGenerativeAI   
    structured_output_models: Dict[str, Any] 


TOOL_MAP = { 
    "product_information": submit_product_information,
    "medical_treatment": submit_medical_treatment,
    "high_level_overview": submit_high_level_overview,
    "claims_made": submit_claims_made,
    "businesses_entities": submit_businesses_entities,
} 

async def return_tool_call_dict(tool_response, tool_map): 
    additional_kwargs = tool_response.additional_kwargs   
    function_to_call = additional_kwargs.get("function_call", None).get("name", None)  
    arguments = additional_kwargs.get("function_call", None).get("arguments", None)    

    tool_to_call = tool_map.get(function_to_call, None)   

    arguments_loaded = json.loads(arguments) 

    return (tool_to_call, arguments_loaded)






initial_state: TranscriptIngestionState = {  
    "input": "Begin the Transcript Chain",   
    "aggregate_summary": "",
    "final_summary": "",
    "summary_prompts_dict": SUMMARY_PROMPTS, 
    "structured_output_prompts_dict": STRUCTURED_EXTRACTOR_PROMPTS,
    "full_transcript": Document(page_content=transcript_text),
    "product_information": "",
    "medical_treatment": "",
    "high_level_overview": "",
    "claims_made": "",
    "businesses_entities": "", 
    "structured_tool_call_dict": {},
    "structured_output_dict": MODEL_MAP,
    "structured_output_tool_dict": TOOL_MAP,
    "anthropic_vertex_llm": anthropic_vertex,
    "google_llm": llm,
    "structured_output_models": {   
        "product_information": llm.bind_tools([submit_product_information]),
        "medical_treatment": llm.bind_tools([submit_medical_treatment]),
        "high_level_overview": anthropic_vertex.bind_tools([submit_high_level_overview]),
        "claims_made": anthropic_vertex.bind_tools([submit_claims_made]),
        "businesses_entities": anthropic_vertex.bind_tools([submit_businesses_entities]),
    },

}




async def generate_summaries(state: TranscriptIngestionState) -> TranscriptIngestionState:  

    full_transcript = state.get("full_transcript", "")   

    summary_prompts_dict = state.get("summary_prompts_dict", {})    


    for summary_key, summary_prompt in summary_prompts_dict.items():   

        prompt_template = PromptTemplate.from_template(summary_prompt)   
        
        chain = prompt_template | state.get("google_llm") | StrOutputParser()   
        response = await chain.ainvoke({"transcript": full_transcript.page_content})   
        state[summary_key] = response   
        state["aggregate_summary"] += f"\n{summary_key}. {response}"   
        
    return {**state}     

async def structured_extraction(state: TranscriptIngestionState) -> TranscriptIngestionState:   

    structured_output_tools = state.get("structured_output_tool_dict", {})    

    structured_output_models = state.get("structured_output_models", {})   

    structured_extractor_prompts_dict = state.get("structured_output_prompts_dict", {})    

    for key, tool in structured_output_tools.items(): 
        structured_model = structured_output_models.get(key, None)    
        prompt = structured_extractor_prompts_dict.get(key, "")    


        if structured_model is None or prompt is None:  
            return None    
        
        prompt_template = PromptTemplate.from_template(prompt) 
        if state.get(key, "") == "": continue  

        specific_summary = state.get(key, "")    
        chain = prompt_template | structured_model     
        response = await chain.ainvoke({"summary": specific_summary})    

        tool_to_call, arguments_loaded = await return_tool_call_dict(response, structured_output_tools)  

        state["structured_tool_call_dict"][key] = arguments_loaded  





        return {**state} 




graph = StateGraph(TranscriptIngestionState)
graph.add_node("generate_summaries", generate_summaries)
graph.add_node("structured_extraction", structured_extraction)

graph.add_edge(START, "generate_summaries")
graph.add_edge("generate_summaries", "structured_extraction")
graph.add_edge("structured_extraction", END)

app = graph.compile()





async def run_graph():
    # Stream steps
    async for step in app.astream(initial_state):
        print("STEP KEYS:", list(step.keys()))

    # Get final state
    final_state = await app.ainvoke(initial_state)

    # Print full state
    print("\n=== Full State ===")
    for key, value in final_state.items():
        print(f"\n[{key}]")
        print(value)

    # Print structured output
    print("\n=== Structured Output ===")
    if "structured_tool_call_dict" in final_state:
        for key, value in final_state["structured_tool_call_dict"].items():
            print(f"\n[{key}]")
            print(value)

    # Print summaries
    print("\n=== Individual Summaries ===")
    summary_keys = ["product_information", "medical_treatment", "high_level_overview", 
                   "claims_made", "businesses_entities"]
    for key in summary_keys:
        print(f"\n## {key}")
        print(final_state.get(key, '')[:2000])  # Trim if too long

    # Print aggregate summary
    print("\n=== Aggregate Summary ===")
    print(final_state.get("aggregate_summary", '')[:4000])  # Show first 4000 chars


asyncio.run(run_graph()) 






    

STEP KEYS: ['generate_summaries']


C:\Users\Pinda\AppData\Local\Temp\ipykernel_26256\2605639054.py:124: RuntimeWarning: coroutine 'return_tool_call_dict' was never awaited
  tool_to_call, arguments_loaded = return_tool_call_dict(response, structured_output_tools)


TypeError: cannot unpack non-iterable coroutine object

In [ ]:
product_summary_test = """ 

product_information. Here's a summary of the product information extracted from the transcript:

**1. Exercise With Oxygen Therapy (EWOT) Systems from 1000 Roads**

*   **Description:** Brad Ley, founder of 1000 Roads, developed EWOT systems for home use after discovering the benefits of exercise with oxygen therapy for his own autoimmune conditions and melanoma. The systems aim to make this therapy more accessible and affordable.
*   **Features:**
    *   **Oxygen Concentrator:** Takes ambient air (21% oxygen) and purifies it to 93% oxygen, which is the medical specification.
    *   **Oxygen Reservoir:** A large bag (approximately 6 feet wide and 5.5 feet tall) that holds 1000 liters of oxygen.
    *   **Mask:** A mask with a two-way valve. One valve opens during inhalation of pure oxygen, and the other closes during exhalation, preventing exhaled gases from re-entering the reservoir.
*   **How it works:** The system works by saturating blood plasma with oxygen, allowing it to reach cells that may be oxygen-deprived due to inflammation in the capillaries. It also has an anti-inflammatory effect.
*   **Protocol:**
    *   Wear the mask and engage in 15 minutes of cardiovascular exercise while breathing 93% pure oxygen.
    *   Intensity should be around 70-80% of theoretical max heart rate.
    *   Adjust intensity based on individual fitness level and health condition, starting slowly for those with chronic illnesses.
*   **Benefits:**
    *   **Anti-inflammatory:** Flooding tissues with oxygen reduces inflammation throughout the body.
    *   **Detoxification:** Enhances the body's natural detoxification processes, particularly through the lungs.
    *   **Increased Energy:** Improves energy production by supporting aerobic respiration.
    *   **Improved Athletic Performance:** Increases endurance and the intensity at which one can work out.
    *   **Recovery:** Helps break down lactic acid and reduces muscle soreness.
    *   **Clear Headedness:** Many users report clearing of brain fog.
*   **Target users:** People with chronic illness, high performers, and athletes.
*   **Pricing:** Brad aimed to significantly lower the price compared to existing EWOT systems, making it more accessible for home use.
*   **Future Directions:** Focus on making it more available and portable.

""" 


TOOL_MAP  



structured_test = llm.bind_tools([TOOL_MAP["product_information"]]) 


prompt_template = PromptTemplate.from_template("Extract all the product information from the following summary. Use you tools to do the extraction. Here is the summary \n {summary}") 



structure_response = structured_test.invoke(prompt_template.format(summary=product_summary_test))  


print(structure_response)  

print(structure_response.additional_kwargs)












content='' additional_kwargs={'function_call': {'name': 'submit_product_information', 'arguments': '{"products": [{"name": "Exercise With Oxygen Therapy (EWOT) Systems from 1000 Roads", "benefits_as_stated": ["Anti-inflammatory: Flooding tissues with oxygen reduces inflammation throughout the body.", "Detoxification: Enhances the body\'s natural detoxification processes, particularly through the lungs.", "Increased Energy: Improves energy production by supporting aerobic respiration.", "Improved Athletic Performance: Increases endurance and the intensity at which one can work out.", "Recovery: Helps break down lactic acid and reduces muscle soreness.", "Clear Headedness: Many users report clearing of brain fog."], "description": "Brad Ley, founder of 1000 Roads, developed EWOT systems for home use after discovering the benefits of exercise with oxygen therapy for his own autoimmune conditions and melanoma. The systems aim to make this therapy more accessible and affordable. The system 

In [30]:
import json  
print(structure_response.additional_kwargs) 

print(structure_response.additional_kwargs.get("function_call", None).get("name", None))  

arguments = structure_response.additional_kwargs.get("function_call", None).get("arguments", None)

submit_product = structure_response.additional_kwargs.get("function_call", None).get("name", None)  


arguments_loaded = json.loads(arguments)

print(arguments_loaded["products"])  



async def return_tool_call_dict(tool_response, tool_map): 
    additional_kwargs = tool_response.additional_kwargs   
    function_to_call = additional_kwargs.get("function_call", None).get("name", None)  
    arguments = additional_kwargs.get("function_call", None).get("arguments", None)    

    tool_to_call = tool_map.get(function_to_call, None)   

    arguments_loaded = json.loads(arguments) 

    return (tool_to_call, arguments_loaded)



# Convert the loaded arguments into ProductSummary model
product_items = []
for product in arguments_loaded["products"]:
    product_item = ProductItem(
        name=product.get("name"),
        description=product.get("description"),
        features=product.get("features", []),
        protocols=product.get("protocols", []),
        benefits_as_stated=product.get("benefits_as_stated", []),
        attribution=[]  # Empty list since no attribution data in input
    )
    product_items.append(product_item)

product_summary = ProductSummary(products=product_items)

print("\nValidated Pydantic Model:")
print(product_summary.model_dump_json(indent=2))





{'function_call': {'name': 'submit_product_information', 'arguments': '{"products": [{"name": "Exercise With Oxygen Therapy (EWOT) Systems from 1000 Roads", "benefits_as_stated": ["Anti-inflammatory: Flooding tissues with oxygen reduces inflammation throughout the body.", "Detoxification: Enhances the body\'s natural detoxification processes, particularly through the lungs.", "Increased Energy: Improves energy production by supporting aerobic respiration.", "Improved Athletic Performance: Increases endurance and the intensity at which one can work out.", "Recovery: Helps break down lactic acid and reduces muscle soreness.", "Clear Headedness: Many users report clearing of brain fog."], "description": "Brad Ley, founder of 1000 Roads, developed EWOT systems for home use after discovering the benefits of exercise with oxygen therapy for his own autoimmune conditions and melanoma. The systems aim to make this therapy more accessible and affordable. The system works by saturating blood pla